### Ingesting to graph from pgdb

In [1]:
# from neo import NeoGraph
# from db import Db

# import pandas as pd

# g = NeoGraph()

# db = Db()
# eng = db.create_engine()
# df = pd.read_sql("""select * from words""", eng)

# df['len'] = df['characters'].apply(lambda x: len(list(x)))
# df['c1'] = df['characters'].apply(lambda x: list(x)[0])
# df['c2'] = df['characters'].apply(lambda x: list(x)[1] if len(x) >= 2 else None)
# df['p1'] = df['pinyin'].apply(lambda x: x.split(' ')[0])
# df['p2'] = df['pinyin'].apply(lambda x: x.split(' ')[1] if ' ' in x else None)
# df['english'] = df['english'].apply(lambda x: x.replace("'", "\\'"))
# df['english_short'] = df['english'].apply(lambda x: x.split(';')[0])
# df['descr'] = df.apply(lambda x: f"""{x['pinyin']} | {x['english_short']}""",axis = 1)

# helper = []
# chars = []
# for i, c in df.iterrows():
#     for h, p in zip(list(c['characters']), c['pinyin'].split(' ')):
#         helper.append(f'{h} : {p}')
#         chars.append(h)

# g.add_characters(list(set(chars)), df)

# g.create_links(df[df['len'] == 2])

In [ ]:
from neo import NeoGraph
from db import Db

g = NeoGraph()

import json

In [24]:
def generate_vis_json_from_char(char='上')

    out_links = g.run_query(f"""match path = (source:Character)-[link:WORD]->(target:Character)
    where source.strokes = '{char}'
    return source,link,target""")

    in_links = g.run_query(f"""match path = (target:Character)<-[link:WORD]-(source:Character)
    where target.strokes = '{char}'
    return source,link,target""")

    root_node = g.run_query(f"""match (n:Character)
    where n.strokes = '{char}'
    return distinct n""")

    connected_nodes = g.run_query(f"""MATCH (n {{strokes : '{char}'}})-[:WORD]-(connected)
    RETURN distinct connected""")

    nodes = [node['n'] for node in root_node]  + [node['connected'] for node in connected_nodes]

    nodesj = json.loads(json.dumps(nodes))

    nodes_plt = nodesj.copy()

    for x in nodes_plt:
        x['id'] = x.pop('strokes')
        x['label'] = x.get('id')
        x.pop('descr', None)

    nodes_out = nodes_plt.copy()

    linksj = json.loads(json.dumps(out_links)) + json.loads(json.dumps(in_links))

    links_plt = []

    for link in linksj:
        links_plt.append({'from':link['source']['strokes'], 'to':link['target']['strokes'], 'label' : link['link']['descr']})

    out = {'nodes':nodes_out, 'edges':links_plt}

    with open(f'./assets/chars/{char}.json', 'w') as outfile:
        json.dump(out, outfile)